# Homodyne tomography

In [ ]:
# Importing libraries
import numpy as np
import scipy as sp
import pandas as pd
from qutip import Qobj, wigner
import os
from pathlib import Path
import sys
import json
from scipy.fft import rfft, irfft, rfftfreq
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from matplotlib import animation
from IPython.display import HTML

# Importing functions
from utils import *

from iMLE import *

from BME import *

# Global plot style settings
plt.rcParams.update({
    "font.family": "serif",
    "font.serif": ["DejaVu Serif"],
    "mathtext.fontset": "cm",            # Use Computer Modern for math
    "font.size": 14,                     # Base font size
    "axes.labelsize": 16,                # Axis label font size
    "axes.titlesize": 16,                # Title font size
    "legend.fontsize": 13,               # Legend font size
    "xtick.labelsize": 13,               # X tick label size
    "ytick.labelsize": 13,               # Y tick label size
    "axes.linewidth": 1.2,               # Thicker axis lines
    "xtick.direction": "in",             # x-yick direction
    "ytick.direction": "in",             # y-tick direction
    "text.usetex": False,                # Enable LaTeX if needed
    "figure.dpi": 100,                   # Good resolution for screens
    "savefig.dpi": 300                   # High resolution for saving
})

# Define function for nice plotting
def PlotSettings(ax, gridlines=False, minimalist=False):
    # Minor ticks
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    # Minimalist style
    if minimalist:
        # Hide top and right spines (borders)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        # Ticks only on bottom and left
        ax.tick_params(which='both', top=False, right=False)
    else:
        # Tick parameters
        ax.tick_params(which='both', direction='in', top=True, right=True)
        ax.tick_params(which='major', length=7, width=1.2)
        ax.tick_params(which='minor', length=4, width=1)
    # Optional grid
    if gridlines:
        ax.grid(True, which="major", linestyle="--", linewidth=0.6, alpha=0.7)
        ax.grid(True, which="minor", linestyle=":", linewidth=0.5, alpha=0.4)

### What is Homodyne tomography?

Homodyne tomography is a method for reconstructing quantum states of light by measuring field quadratures $Q_\theta$ at different local oscillator (LO) phases $\theta$ using a Homodyne detection scheme. From these measurements, one can recover the Wigner function $W(Q,P)$ and density matrix $\rho$, which makes it a key tool for verifying the preparation of nonclassical states in quantum optics.

<img src="../images/HomodyneTomography.png" alt="Homodyne Tomography" height="250"/>
<img src="../images/HomodyneDetector.png" alt="Homodyne Detector" height="250"/>

In the paper *Optical Continuous-Variable Qubit* (see references), superpositions of squeezed vacuum and squeezed single-photon states were engineered using photon subtraction. The setup employed an optical parametric oscillator (OPO) to generate squeezed vacuum, a small fraction of which was tapped off, displaced, and detected by an avalanche photodiode (APD). Conditional detection prepared the desired superpositions, which were then analyzed using balanced homodyne detection across multiple LO phases.

<img src="../images/ExperimentalSetup.png" alt="Homodyne Tomography" height="500"/>

In this notebook, we work with the experimental data obtained using this setup. Our goal is to benchmark and compare two different reconstruction methods - Maximum Likelihood Estimation (MLE) and Bayesian Mean Estimation (BME) - to reproduce the Wigner functions reported in the paper.

### Loading and processing data

We start by downloading and un-zipping the data from the dropbox link bellow:

https://www.dropbox.com/scl/fi/0cb9i9vx9w0b1lpro8wpq/data-tora.zip?rlkey=nkouczz7r9ylnmkd3n639z7kt&dl=0

After downloading and un-zipping, put the **data-tora** folder into the **data** folder in the repo. The folder contains 3 different dates **091027**, **091028** and **091029** each containing a collection of measurements of different states - like **cat1** or **tora5**.
For each state 30,000 measurements were performed at the angles 0, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, as well as a vacuum measurement for reference.

The data was acquired by a LeCroy HDO6034 oscilloscope. Since it is in a proprietary binary format, we use the script **lecroy.py** to import it. Based on the folder and file structure of the **data-tora** folder, we created a script for processing the data collecting all the traces corresponding to different phases into a single .npy file for each state.

In [ ]:
# Try if it runs, otherwise manually run the preprocessing script
#parent = str(Path.cwd().parent)
#%run preprocess_data.py {parent}

**Update: Do Not Download File from dropbox. We have included the .npy and .csv files.**

In [ ]:
parent = Path(os.path.dirname(os.getcwd()))
date = "091027"
state = "cat2"

data_path = parent / "data" / "processed_data" / date
data_raw = np.load(data_path / (state + '.npy'))

dt = json.load(open(data_path / 'dts.json'))[state]
N = data_raw.shape[2]
t = np.linspace(0, dt*N, N, endpoint=False)

In [ ]:
data_raw.shape

The structure of the .npy file is (n_angles, n_traces, n_measurements). Bellow we plot a single trace.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(t*1e9, data_raw[0, 0, :])
ax.set_xlabel("Time (ns)")
ax.set_ylabel("Amplitude")
ax.set_xlim(0, t[-1]*1e9)
PlotSettings(ax, gridlines=True, minimalist=True)
plt.show()

### Calculating quadrature values

The OPO used in the experiment outputs a continuum of temporal modes, however the conditioned quantum state only occupies a single temporal mode, so we need to perform temporal filtering by selecting a mode function $f_s(t)$ to integrate over, giving us a single quadrature value as
$$
Q_\theta=\int_{-\infty}^\infty f_s(t)Q_\theta(t)dt
$$
The mode function choosen here is $f_s(t)=\sqrt{\gamma}e^{-\gamma|t-t_0|}$, where $\gamma$ is the total decay rate of the OPO - corresponding to the HWHM (half-widthhalf-maximum) bandwidth of the approximately Lorentzian shaped resonances, and $t_0$ is the exact time a photon is subtracted from the state. The total decay rate is calculated as $\gamma=2\pi f$, with $f=9$ MHz being the bandwidth of the OPO. The time $t_0$ can be estimated by calculating the variance over all the traces for a given angle and then finding the maximum of the peak. This can be seen on the plots bellow.

In [ ]:
# Making a figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
#fig.subplots_adjust(wspace=0.15)

# Subplot 1
for i in range(12):
    state = data_raw[i,:,:]
    state_var = state.var(axis=0)
    ax1.plot(t*1e9, state_var, label=f'{15*i} deg')
# Make legend in 2 columns
ax1.legend(ncol=2)
# Set title and labels
ax1.set_title("Variance of different angles")
ax1.set_xlabel("Time (ns)")
ax1.set_ylabel("Variance")
PlotSettings(ax1, gridlines=True, minimalist=True)

# Subplot 2
# Extract vacuum data
vacuum = data_raw[-1,:,:]
# Define temporal mode from peak in variance
temporal_mode = fs(t, t[39])  # shape (T,)
# Plot
ax2.plot(t*1e9, temporal_mode)
# Set title and labels
ax2.set_title("Extracted Temporal Mode")
ax2.set_xlabel("Time (ns)")
ax2.set_ylabel("Amplitude")
PlotSettings(ax2, gridlines=True, minimalist=True)

plt.tight_layout()
plt.show()

By automating this procedure, we can process all the .npy arrays and make dataframes in .csv files which contain only the quadrature values for the 12 different angles. These values are normalized with respect to the vacuum.

In [ ]:
# Loading data
parent = Path(os.path.dirname(os.getcwd()))
date = "091027"
state = "cat2"

data_path = parent / "data" / "dataframes" / date
data_quadratures = pd.read_csv(data_path / (state + '.csv'))
data_quadratures

In [ ]:
# Convert to numpy array
x_values = np.array(data_quadratures)
x_values = np.swapaxes(x_values, 0, 1)
# Make array from 0 to 165 in steps of 15
thetas = np.arange(0, 166, 15)
# Convert to radians
thetas = np.radians(thetas)
# Put offset on values
x0 = -0.08
theta0 = np.deg2rad(147)

In [ ]:
for i in range(12):
    plt.hist(x_values[i], bins=100, density=False, alpha=0.4, label=f"{15*i} deg")

plt.legend(fontsize=12)
plt.title("Calibrated Quadrature Distributions")
plt.xlabel("Quadrature Value, $X_\\theta$")
plt.ylabel("Counts")
plt.grid(True)
plt.show()

### Reconstruction algoritms

In the following reconstruction algorithms, the goal is to reconstruct the unknown quantum state $\hat{\rho}$ given a set of quadrature measurements $\{x_i\}$ and LO phase angles $\{\theta_j\}$. Each measurement outcome corresponds to projecting the state onto a quadrature eigenstate $|\theta_j,x_i\rangle$, and the probability of obtaining that outcome is given by
\begin{equation}
P(x_i;\theta_j)=\langle\theta_j,x_i|\hat{\rho}|\theta_j,x_i\rangle \tag{1}
\end{equation}
To estimate the density matrix that best fits the measured data, one needs to maximize the likelihood function
\begin{equation}
\mathcal{L(\hat{\rho})}=\prod_{i,j}P(x_i;\theta_j)^{c_i}, \tag{2}
\end{equation}
where $c_i$ is the number of times a particular outcome occurs. In practice, one maximizes the log-likelihood
\begin{equation}
\ln{\mathcal{L(\hat{\rho})}}=\sum_{i,j} c_i\ln P(x_i;\theta_j) \tag{3}
\end{equation}
To calculate the probabilities $P(x_i;\theta_j)$ given by eq. (1), one can express the quadrature eigenstates in the Fock (or photon number) basis as
\begin{equation}
|\theta_j,x_i\rangle=\sum_{n=0}^{N-1}\langle n|\theta_j,x_i\rangle|n\rangle, \tag{4}
\end{equation}
where the overlap between the number and quadrature eigenstates is given by the well known stationary solution of the Schrödinger equation for a particle in a harmonic potential
\begin{equation}
\langle n|\theta,x\rangle=e^{-in\theta}\left(\frac{1}{\pi}\right)^{1/4}\frac{H_n(x)}{\sqrt{2^nn!}}\exp(-x^2/2) \tag{5}
\end{equation}

#### Itterative Maxium Likelihood Estimation (iMLE)

In Maximum Likelihood estimation, the goal is to find the density matrix $\hat{\rho}$ which maximizes the likelihood $\mathcal{L}$ given by eq. (2). To do so, one introduces the iteration operator
\begin{equation}
\hat{R}(\hat{\rho})=\sum_{i,j}\frac{c_i}{P(x_i;\theta_j)} |\theta_j,x_i\rangle\langle\theta_j,x_i| \tag{6}
\end{equation}
Looking at eq. (2) one can see that by inserting the quantum state $\rho_0$ which is most likely to produce the observed data set, we must have $c_i\propto P(x_i;\theta_j)$. Furthermore, since $\sum_{i,j}|\theta_j,x_i\rangle\langle\theta_j,x_i|\propto\hat{1}$, we find $\hat{R}(\hat{\rho_0})\propto\hat{1}$ and thus
$$
\hat{R}(\hat{\rho_0})\hat{\rho_0}\propto\hat{\rho_0}\hat{R}(\hat{\rho_0})\propto\hat{\rho_0}\Rightarrow\hat{R}(\hat{\rho_0})\hat{\rho_0}\hat{R}(\hat{\rho_0})\propto\hat{\rho_0}
$$
The last relation forms the basis for the iterative algorithm. We start by choosing some initial density matrix e.g. $\hat{\rho}^{(0)}=\mathcal{N}[\hat{1}]$, where $\mathcal{N}$ is a normalization to a unitary trace and then apply repetitive itterations over $k$
$$
\hat{\rho}^{(k+1)}=\mathcal{N}[\hat{R}(\hat{\rho}^{(k)})\hat{\rho}^{(k)}\hat{R}(\hat{\rho}^{(k)})],
$$
each time constructing a new itteration operator as given by eq. (6). The total number of itterations is set by monitoring the likelihood, which will monotonically increase, and then determine when the changes are small enough.

The cell bellow runs the algoritm on the data and plots the change in log-likelihood over itterations

In [ ]:
# Running iMLE with N=15 and num_bins=400
rhos, likelihoods = run_iMLE(thetas-theta0, x_values-x0, N=15,
                             num_bins=400, max_iters=100, tol=1e-1)
# Determine total number of measurements and calculate delta log-likelihood per sample
n_samples = x_values.size
per_sample = np.array(likelihoods) / n_samples
delta_ll = per_sample - np.max(per_sample)
iterations = np.arange(0, len(delta_ll), 1)

In [ ]:
# Making a figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
#fig.subplots_adjust(wspace=0.15)

# Subplot 1
ax1.plot(iterations[:10], delta_ll[:10], marker="o", lw=1.5)
ax1.set_xlabel("Iteration")
ax1.set_ylabel("Δ log-likelihood per sample")
ax1.set_title("Convergence of iMLE")
PlotSettings(ax1, gridlines=True, minimalist=True)
#ax1.grid(True, alpha=0.3)

# Subplot 2
ax2.plot(iterations[10:], delta_ll[10:], marker="o", lw=1.5)
ax2.set_xlabel("Iteration")
#ax2.set_ylabel("Log-Likelihood")
ax2.set_title("Convergence of iMLE")
PlotSettings(ax2, gridlines=True, minimalist=True)

plt.tight_layout()
plt.show()

In [ ]:
# Grid and parameters
xvec = np.linspace(-4, 4, 200)
X, Y = np.meshgrid(xvec, xvec)
vmin, vmax = -1/np.pi, 1/np.pi
levels = np.linspace(vmin, vmax, 41)

# Precompute Wigner functions with QuTiP
Ws = [wigner(Qobj(rho), xvec, xvec) for rho in tqdm(rhos, desc="Computing Wigner functions")]

# Precompute the diagonal elements
diagonals = [np.diag(rho).real for rho in rhos]

N = rhos[-1].shape[0]
# Make array from 0 to N in steps of 1
n_values = np.arange(0, N, 1)

In [ ]:
# Making a figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(
    1, 2, figsize=(14, 6),
    gridspec_kw={'width_ratios': [1.5, 1]}
)
fig.subplots_adjust(wspace=0.15)

# Subplot 1
ax1.set_title('Wigner function')
# Filled contours with transparency
contour_filled = ax1.contourf(X, Y, Ws[-1], levels=levels,
                            cmap='seismic', alpha=0.8, vmin=vmin, vmax=vmax)
# Colorbar
cbar = fig.colorbar(contour_filled, ax=ax1, ticks=[-1/np.pi, 1/np.pi])
cbar.ax.set_yticklabels([r"$-1/\pi$", r"$1/\pi$"])  # custom tick labels if needed
# Move axes to cross at (0,0)
ax1.spines['left'].set_position('zero')
ax1.spines['bottom'].set_position('zero')
ax1.spines['right'].set_color('none')
ax1.spines['top'].set_color('none')
# Set x- and y-ticks skipping 0
ax1.set_xticks([-3, -2, -1, 1, 2, 3])
ax1.set_yticks([-3, -2, -1, 1, 2, 3])
PlotSettings(ax1, minimalist=True)

# Subplot 2
ax2.set_title('Photon statistics')

for i in range(len(n_values)):
    ax2.vlines(n_values[i], 0, diagonals[-1][i], colors='blue', linestyles='dashed')

ax2.plot(n_values, diagonals[-1], 'o', color='blue')
#ax2.hist(diagonals[-1], bins=len(diagonals[-1]), alpha=0.7)
ax2.set_xlabel('Fock state |n>')
ax2.set_ylabel('Probability')
ax2.set_xticks(n_values)
ax2.set_ylim(0, 1)
ax2.spines[['right', 'top']].set_visible(False)

#plt.tight_layout()
plt.show()

In [ ]:
# --- Figure setup ---
fig, (ax1, ax2) = plt.subplots(
    1, 2, figsize=(14, 6),
    gridspec_kw={'width_ratios': [1.5, 1]}
)
fig.subplots_adjust(wspace=0.15)

# Initial contour on ax1
contour = ax1.contourf(X, Y, Ws[0], levels=levels,
                       cmap="seismic", vmin=vmin, vmax=vmax, alpha=0.8)
cbar = fig.colorbar(contour, ax=ax1, ticks=[-1/np.pi, 1/np.pi])
cbar.ax.set_yticklabels([r"$-1/\pi$", r"$1/\pi$"])

# Ax1 styling
ax1.spines['left'].set_position('zero')
ax1.spines['bottom'].set_position('zero')
ax1.spines['right'].set_color('none')
ax1.spines['top'].set_color('none')
ax1.set_xticks([-3, -2, -1, 1, 2, 3])
ax1.set_yticks([-3, -2, -1, 1, 2, 3])

# Ax2 styling
ax2.set_xlabel('Fock state |n>')
ax2.set_ylabel('Probability')
ax2.set_xticks(n_values)
PlotSettings(ax2, minimalist=True)

# --- Animation functions ---
def init():
    return []

def animate(i):
    # --- ax1: update Wigner contour ---
    while ax1.collections:
        ax1.collections[0].remove()
    ax1.set_title(f"Wigner Function (Iteration {i})")
    contour = ax1.contourf(X, Y, Ws[i], levels=levels,
                           cmap="seismic", vmin=vmin, vmax=vmax, alpha=0.8)

    # --- ax2: clear and redraw ---
    ax2.clear()
    ax2.set_xlabel('Fock state |n>')
    ax2.set_ylabel('Probability')
    ax2.set_xticks(n_values)
    ax2.set_ylim(0, 1)
    for j, n in enumerate(n_values):
        ax2.vlines(n, 0, diagonals[i][j], colors='blue', linestyles='dashed')
    ax2.plot(n_values, diagonals[i], 'o', color='blue')

    return [contour]  # returning ax2 artists is optional when blit=False

# --- Create animation ---
ani = animation.FuncAnimation(fig, animate, init_func=init,
                              frames=len(rhos), interval=100, blit=False)

# --- Display in notebook ---
html_anim = HTML(ani.to_jshtml())
plt.close(fig)
display(html_anim)

#### Bayesian Mean Estimation (BME)

BME is a Bayesian approach to estimating parameters of a probability distribution. Here’s the idea step by step, as explained in [X] [this paper](https://iopscience.iop.org/article/10.1088/1367-2630/12/4/043034/pdf):

1. Use the data to generate a likelihood function, $\mathcal{L}(\rho) = p(M|\rho)$. $\mathcal{L}$ is not a probability
distribution; it quantifies the relative plausibility of different state assignments.

2. Choose a prior distribution over states, $\pi_{0}(\rho)d\rho$. It represents the estimator’s ignorance,
and should generally be chosen to be as ‘uniform’, or uninformative, as possible.

3. Multiply the prior by the likelihood and normalize to obtain a posterior distribution

$$
\pi_{f}(\rho)d\rho \propto \mathcal{L}(\rho)\pi_{0}(\rho)d\rho
$$ 

which represents the estimator’s knowledge. The proportionality constant is set by normalization.

4. Report the mean of this posterior,

$$
\hat\rho_{BME} = \int \rho \pi_{f}(\rho)d\rho
$$

This is the best concise description of the estimator’s knowledge.

The main goal is to find a Markoc chain of density matrices that, when averaged, gives the posterior mean $\hat\rho_{BME}$. There are multiple algorithms to build this chain, but in this project we have implemented the Metropolis-hastings method. This works as follows:
1. Start with an initial guess for the density matrix $\rho^{(0)}$. In our case, we have assumed $\rho^{(0)} = \frac{I}{N}$, the maximally mixed state.
2. Propose a new density matrix $\rho'$ by perturbing the current state $\rho$ with a small random step. This is done by Cholesky parametrization, where we decompose the density matrix into a lower triangular matrix T such that 
$$
\rho = \frac{T^\dagger T}{\mathrm{Tr}(T^\dagger T)}
$$
Then, we perturb T by adding a small (multiplied by a factor $\epsilon$) random (sampled from a $\mathcal{N}(0,1)$) matrix $\Delta T$, and the resulting new density matrix is given by:
$$
\rho' = \frac{(T + \epsilon \Delta T)^\dagger (T + \epsilon \Delta T)}{\mathrm{Tr}((T + \epsilon \Delta T)^\dagger (T + \epsilon \Delta T))}
$$

3. Calculate the acceptance probability $\alpha = \min\left(1, \frac{P(\rho')}{P(\rho)}\right)$. In our case, we use log $\alpha$ instead, which is given by:
$$
\log \alpha = (\log \mathcal{L}(\rho') + \log \pi_0(\rho')) - (\log \mathcal{L}(\rho) - \log \pi_0(\rho))
$$

4.  Generate a random number $r$ from a uniform distribution $U(0,1)$, calculate its logarithm, and compare it to $\log \alpha$. If $\log r < \log \alpha$, accept the new state $\rho'$, otherwise keep the current state $\rho$.

5. Repeat steps 2-4 for as many iterations as needed to build a Markov chain of density matrices.

As part of the last steps, we get rid of the first 20% of the chain (burn-in) since they strongly depend on the initial guess $\rho^{(0)}$, and average the remaining states to get the final estimate $\hat\rho_{BME}$. The log-likelihood is also averaged over the same chain.

Without further ado, let's see how our BME reconstruction method performs on the data.

In [ ]:
N = 10
nrho = 5000
num_bins = 400
eps=0.01

rho_est, logL_chain = run_BME(thetas-theta0, 
              x_values-x0,
              num_bins=num_bins,
              nrho=nrho,
              N=N,
              epsilon=eps
              )

In [ ]:
# Grid and parameters
xvec = np.linspace(-4, 4, 200)
X, Y = np.meshgrid(xvec, xvec)
vmin, vmax = -1/np.pi, 1/np.pi
levels = np.linspace(vmin, vmax, 41)

# Precompute Wigner functions with QuTiP
Ws = [wigner(Qobj(rho_est), xvec, xvec)]

# Precompute the diagonal elements
diagonals = [np.diag(rho_est).real]

N = rho_est.shape[0]
# Make array from 0 to N in steps of 1
n_values = np.arange(0, N, 1)

In [ ]:
# Making a figure with 2 subplots
fig, (ax1, ax2) = plt.subplots(
    1, 2, figsize=(14, 6),
    gridspec_kw={'width_ratios': [1.5, 1]}
)
fig.subplots_adjust(wspace=0.15)

# Subplot 1
ax1.set_title('Wigner function')
# Filled contours with transparency
contour_filled = ax1.contourf(X, Y, Ws[-1], levels=levels,
                            cmap='seismic', alpha=0.8, vmin=vmin, vmax=vmax)
# Colorbar
cbar = fig.colorbar(contour_filled, ax=ax1, ticks=[-1/np.pi, 1/np.pi])
cbar.ax.set_yticklabels([r"$-1/\pi$", r"$1/\pi$"])  # custom tick labels if needed
# Move axes to cross at (0,0)
ax1.spines['left'].set_position('zero')
ax1.spines['bottom'].set_position('zero')
ax1.spines['right'].set_color('none')
ax1.spines['top'].set_color('none')
# Set x- and y-ticks skipping 0
ax1.set_xticks([-3, -2, -1, 1, 2, 3])
ax1.set_yticks([-3, -2, -1, 1, 2, 3])
PlotSettings(ax1, minimalist=True)

# Subplot 2
ax2.set_title('Photon statistics')

for i in range(len(n_values)):
    ax2.vlines(n_values[i], 0, diagonals[-1][i], colors='blue', linestyles='dashed')

ax2.plot(n_values, diagonals[-1], 'o', color='blue')
ax2.set_xlabel('Fock state |n>')
ax2.set_ylabel('Probability')
ax2.set_xticks(n_values)
ax2.set_ylim(0, 1)
ax2.spines[['right', 'top']].set_visible(False)

#plt.tight_layout()
plt.show()

### Benchmarks and comparisons

In [ ]:
N_values = [i for i in range(3, 11)]
nbin_values = [i*20 for i in range(3, 11)]

delta_ll_mle, runtime_grid_mle = run_iMLE_benchmark(
    thetas, x_values, N_values, nbin_values,
    tol=1e-2, max_iters=1000
)

delta_ll_bme, runtime_grid_bme = run_BME_benchmark(
    thetas, x_values, N_values, nbin_values,[5000],
)

In [ ]:
plot_comparison(
    delta_ll_mle, delta_ll_bme,
    runtime_grid_mle, runtime_grid_bme,
    N_values, nbin_values
)

In [ ]:
N_values = [i for i in range(3, 11)]
nbin_values = [i*20 for i in range(3, 11)]

delta_ll_mle_2, runtime_grid_mle_2 = run_iMLE_benchmark(
    thetas, x_values, N_values, nbin_values,
    tol=1e-3, max_iters=1000
)

delta_ll_bme_2, runtime_grid_bme_2 = run_BME_benchmark(
    thetas, x_values, N_values, nbin_values,[10000],
)

In [ ]:
plot_comparison(
    delta_ll_mle_2, delta_ll_bme_2,
    runtime_grid_mle_2, runtime_grid_bme_2,
    N_values, nbin_values
)